In [ ]:
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 1,
   "id": "c22687dd-2b31-460f-8d34-1bbf0851675f",
   "metadata": {},
   "outputs": [
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "2023-03-13 17:42:31.433495: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA\n",
      "To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.\n",
      "2023-03-13 17:42:32.293323: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64\n",
      "2023-03-13 17:42:32.293442: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64\n",
      "2023-03-13 17:42:32.293454: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.\n"
     ]
    }
   ],
   "source": [
    "import numpy as np\n",
    "import json\n",
    "import tensorflow.keras as K\n",
    "import tensorflow as tf\n",
    "import cv2\n",
    "from PIL import Image"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 2,
   "id": "a7bcd647-473d-49f3-82c4-1c0212525783",
   "metadata": {},
   "outputs": [],
   "source": [
    "import pandas as pd\n",
    "import json\n",
    "\n",
    "with open('data/malaria/training.json') as f_in:\n",
    "    data_train = json.load(f_in)\n",
    "\n",
    "with open('data/malaria/test.json') as f_in:\n",
    "    data_test = json.load(f_in)\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 3,
   "id": "3577ddbd-17b0-4ae2-8380-6b39640f78a0",
   "metadata": {},
   "outputs": [],
   "source": [
    "cells_train_df = {\n",
    "    'path':[],\n",
    "    'min_r':[],\n",
    "    'min_c':[],\n",
    "    'max_r':[],\n",
    "    'max_c':[],\n",
    "    'r_len':[],\n",
    "    'c_len':[],\n",
    "    'category':[]\n",
    "}\n",
    "for image in data_train:    \n",
    "    #Populate cells_df dict\n",
    "    for box in image['objects']:\n",
    "        cells_train_df['path'].append(image['image']['pathname'])\n",
    "        cells_train_df['min_r'].append(box['bounding_box']['minimum']['r'])\n",
    "        cells_train_df['min_c'].append(box['bounding_box']['minimum']['c'])\n",
    "        cells_train_df['max_r'].append(box['bounding_box']['maximum']['r'])\n",
    "        cells_train_df['max_c'].append(box['bounding_box']['maximum']['c'])\n",
    "        cells_train_df['r_len'].append(box['bounding_box']['maximum']['r']-box['bounding_box']['minimum']['r'])\n",
    "        cells_train_df['c_len'].append(box['bounding_box']['maximum']['c']-box['bounding_box']['minimum']['c'])\n",
    "        cells_train_df['category'].append(box['category'])"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 4,
   "id": "f8f40646-00b9-4b93-8aa2-b36f361a302b",
   "metadata": {},
   "outputs": [],
   "source": [
    "cells_test_df = {\n",
    "    'path':[],\n",
    "    'min_r':[],\n",
    "    'min_c':[],\n",
    "    'max_r':[],\n",
    "    'max_c':[],\n",
    "    'r_len':[],\n",
    "    'c_len':[],\n",
    "    'category':[]\n",
    "}\n",
    "for image in data_test:    \n",
    "    #Populate cells_df dict\n",
    "    for box in image['objects']:\n",
    "        cells_test_df['path'].append(image['image']['pathname'])\n",
    "        cells_test_df['min_r'].append(box['bounding_box']['minimum']['r'])\n",
    "        cells_test_df['min_c'].append(box['bounding_box']['minimum']['c'])\n",
    "        cells_test_df['max_r'].append(box['bounding_box']['maximum']['r'])\n",
    "        cells_test_df['max_c'].append(box['bounding_box']['maximum']['c'])\n",
    "        cells_test_df['r_len'].append(box['bounding_box']['maximum']['r']-box['bounding_box']['minimum']['r'])\n",
    "        cells_test_df['c_len'].append(box['bounding_box']['maximum']['c']-box['bounding_box']['minimum']['c'])\n",
    "        cells_test_df['category'].append(box['category'])"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 5,
   "id": "4bfbad5c-ca86-4d48-adeb-2dd22de66b30",
   "metadata": {},
   "outputs": [],
   "source": [
    "cells_train_df = pd.DataFrame(cells_train_df)\n",
    "cells_test_df = pd.DataFrame(cells_test_df)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 6,
   "id": "f7f7ce39-e762-434b-9cd2-04226c2f95f5",
   "metadata": {},
   "outputs": [],
   "source": [
    "indexes = cells_train_df[cells_train_df.category == 'red blood cell'].index\n",
    "cells_train_df = cells_train_df.drop(indexes)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 7,
   "id": "e14b19eb-5e82-4129-8179-fdf58470313a",
   "metadata": {},
   "outputs": [],
   "source": [
    "indexes = cells_test_df[cells_test_df.category == 'red blood cell'].index\n",
    "cells_test_df = cells_test_df.drop(indexes)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 9,
   "id": "1c8b9856-3e2d-4063-87d8-1f7cef402adc",
   "metadata": {},
   "outputs": [],
   "source": [
    "cells_arr_train = []\n",
    "labels_train = []\n",
    "for index, row in cells_train_df.iterrows():\n",
    "    full_image = cv2.imread(f\"data/malaria{row['path']}\")\n",
    "    min_r, max_r = row['min_r'], row['max_r']\n",
    "    min_c, max_c = row['min_c'], row['max_c']\n",
    "    boxed_image = Image.fromarray(full_image[min_r:max_r, min_c:max_c, :], 'RGB').resize((200,200))\n",
    "    cells_arr_train.append(np.array(boxed_image))\n",
    "    labels_train.append(row['category'])"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 11,
   "id": "c54088a7-35e4-45d6-a3e5-cfede012a0a4",
   "metadata": {},
   "outputs": [],
   "source": [
    "cells_arr_test = []\n",
    "labels_test = []\n",
    "for index, row in cells_test_df.iterrows():\n",
    "    full_image = cv2.imread(f\"data/malaria{row['path']}\")\n",
    "    min_r, max_r = row['min_r'], row['max_r']\n",
    "    min_c, max_c = row['min_c'], row['max_c']\n",
    "    boxed_image = Image.fromarray(full_image[min_r:max_r, min_c:max_c, :], 'RGB').resize((200,200))\n",
    "    cells_arr_test.append(np.array(boxed_image))\n",
    "    labels_test.append(row['category'])"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 12,
   "id": "dca1a126-2318-4305-9b19-c19188b24bb2",
   "metadata": {},
   "outputs": [],
   "source": [
    "X_train = np.array(cells_arr_train)\n",
    "y_train = np.array(labels_train)\n",
    "X_test = np.array(cells_arr_test)\n",
    "y_test = np.array(labels_test)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 15,
   "id": "745c3828-0876-4f00-8b71-c83f56ca5ded",
   "metadata": {},
   "outputs": [],
   "source": [
    "from sklearn.preprocessing import OneHotEncoder\n",
    "def preprocess_data(X, Y):\n",
    "    \"\"\"trains a convolutional neural network to classify the dataset\"\"\"\n",
    "    X_p = K.applications.resnet50.preprocess_input(X)\n",
    "    ohe = OneHotEncoder(sparse=False)\n",
    "    Y_p = ohe.fit_transform(Y.reshape(-1,1))\n",
    "    return X_p, Y_p"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 21,
   "id": "01cf8acf-3cf0-47f9-93d4-79f0fbd7f2d2",
   "metadata": {},
   "outputs": [],
   "source": [
    "trainX = K.applications.resnet50.preprocess_input(X_train)\n",
    "testX = X_p = K.applications.resnet50.preprocess_input(X_test)\n",
    "ohe = OneHotEncoder(sparse=False)\n",
    "trainy = ohe.fit_transform(y_train.reshape(-1,1))\n",
    "testy = ohe.transform(y_test.reshape(-1,1))"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 22,
   "id": "9e15ee80-ab13-4791-a901-bc351db1ad45",
   "metadata": {},
   "outputs": [],
   "source": [
    "inputs = K.Input(shape=(200, 200, 3))\n",
    "#Loading the ResNet50 model with pre-trained ImageNet weights\n",
    "resnet = K.applications.ResNet50(weights='imagenet',include_top=False,input_tensor=inputs)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 23,
   "id": "78bdaa16-2bac-4bb5-898e-b30c1abf2869",
   "metadata": {},
   "outputs": [],
   "source": [
    "for layer in resnet.layers[:170]:\n",
    "   layer.trainable = False"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 24,
   "id": "a5830123-ed0f-4080-9c81-2aad8d59feb8",
   "metadata": {},
   "outputs": [],
   "source": [
    "model = K.models.Sequential()\n",
    "model.add(K.layers.Lambda(lambda x: tf.image.resize(x,(200, 200))))\n",
    "model.add(resnet)\n",
    "model.add(K.layers.GlobalAveragePooling2D())\n",
    "model.add(K.layers.BatchNormalization())\n",
    "model.add(K.layers.Dense(256, activation='relu'))\n",
    "model.add(K.layers.BatchNormalization())\n",
    "model.add(K.layers.Dense(128, activation='relu'))\n",
    "model.add(K.layers.Dropout(0.3))\n",
    "model.add(K.layers.BatchNormalization())\n",
    "model.add(K.layers.Dense(64, activation='relu'))\n",
    "model.add(K.layers.Dropout(0.3))\n",
    "model.add(K.layers.BatchNormalization())\n",
    "model.add(K.layers.Dense(6, activation='softmax'))"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 25,
   "id": "baae7dd8-1a8d-47ec-908a-b821711b172e",
   "metadata": {},
   "outputs": [
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "WARNING:absl:`lr` is deprecated, please use `learning_rate` instead, or use the legacy optimizer, e.g.,tf.keras.optimizers.legacy.RMSprop.\n",
      "2023-03-13 17:47:56.858043: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 1292640000 exceeds 10% of free system memory.\n",
      "2023-03-13 17:47:58.311704: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 1292640000 exceeds 10% of free system memory.\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Epoch 1/10\n",
      "84/85 [============================>.] - ETA: 0s - loss: 0.6315 - accuracy: 0.5104\n",
      "Epoch 1: val_accuracy improved from -inf to 0.72078, saving model to cifar10.h5\n",
      "85/85 [==============================] - 19s 133ms/step - loss: 0.6316 - accuracy: 0.5098 - val_loss: 0.4258 - val_accuracy: 0.7208\n",
      "Epoch 2/10\n",
      "84/85 [============================>.] - ETA: 0s - loss: 0.4253 - accuracy: 0.6670\n",
      "Epoch 2: val_accuracy did not improve from 0.72078\n",
      "85/85 [==============================] - 9s 102ms/step - loss: 0.4252 - accuracy: 0.6669 - val_loss: 0.3764 - val_accuracy: 0.6136\n",
      "Epoch 3/10\n",
      "84/85 [============================>.] - ETA: 0s - loss: 0.2757 - accuracy: 0.7340\n",
      "Epoch 3: val_accuracy did not improve from 0.72078\n",
      "85/85 [==============================] - 9s 103ms/step - loss: 0.2756 - accuracy: 0.7341 - val_loss: 0.3092 - val_accuracy: 0.6266\n",
      "Epoch 4/10\n",
      "84/85 [============================>.] - ETA: 0s - loss: 0.2035 - accuracy: 0.7731\n",
      "Epoch 4: val_accuracy did not improve from 0.72078\n",
      "85/85 [==============================] - 9s 101ms/step - loss: 0.2037 - accuracy: 0.7724 - val_loss: 0.2846 - val_accuracy: 0.6688\n",
      "Epoch 5/10\n",
      "84/85 [============================>.] - ETA: 0s - loss: 0.1682 - accuracy: 0.8106\n",
      "Epoch 5: val_accuracy did not improve from 0.72078\n",
      "85/85 [==============================] - 8s 99ms/step - loss: 0.1684 - accuracy: 0.8106 - val_loss: 0.3124 - val_accuracy: 0.6396\n",
      "Epoch 6/10\n",
      "84/85 [============================>.] - ETA: 0s - loss: 0.1351 - accuracy: 0.8512\n",
      "Epoch 6: val_accuracy did not improve from 0.72078\n",
      "85/85 [==============================] - 8s 98ms/step - loss: 0.1355 - accuracy: 0.8511 - val_loss: 0.5254 - val_accuracy: 0.3864\n",
      "Epoch 7/10\n",
      "84/85 [============================>.] - ETA: 0s - loss: 0.1091 - accuracy: 0.8884\n",
      "Epoch 7: val_accuracy did not improve from 0.72078\n",
      "85/85 [==============================] - 9s 101ms/step - loss: 0.1097 - accuracy: 0.8879 - val_loss: 0.3274 - val_accuracy: 0.6331\n",
      "Epoch 8/10\n",
      "84/85 [============================>.] - ETA: 0s - loss: 0.0827 - accuracy: 0.9234\n",
      "Epoch 8: val_accuracy did not improve from 0.72078\n",
      "85/85 [==============================] - 8s 97ms/step - loss: 0.0835 - accuracy: 0.9224 - val_loss: 0.4170 - val_accuracy: 0.5714\n",
      "Epoch 9/10\n",
      "84/85 [============================>.] - ETA: 0s - loss: 0.0672 - accuracy: 0.9427\n",
      "Epoch 9: val_accuracy did not improve from 0.72078\n",
      "85/85 [==============================] - 8s 97ms/step - loss: 0.0673 - accuracy: 0.9424 - val_loss: 0.3657 - val_accuracy: 0.6006\n",
      "Epoch 10/10\n",
      "84/85 [============================>.] - ETA: 0s - loss: 0.0563 - accuracy: 0.9550\n",
      "Epoch 10: val_accuracy did not improve from 0.72078\n",
      "85/85 [==============================] - 8s 98ms/step - loss: 0.0564 - accuracy: 0.9547 - val_loss: 0.7156 - val_accuracy: 0.3896\n"
     ]
    },
    {
     "data": {
      "text/plain": [
       "<keras.callbacks.History at 0x7fb66c13ae10>"
      ]
     },
     "execution_count": 25,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "model.compile(loss='binary_crossentropy',\n",
    "     optimizer=K.optimizers.RMSprop(lr=2e-5), metrics=['accuracy'])\n",
    "checkpointer = K.callbacks.ModelCheckpoint(filepath='cifar10.h5',\n",
    "     monitor=\"val_accuracy\", verbose=1, save_best_only=True)\n",
    "model.fit(trainX, trainy, batch_size=32, epochs=10, verbose=1,\n",
    "     callbacks=[checkpointer],validation_data=(testX, testy), shuffle=True)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "f3a592ff-2d73-47e8-b73b-48a9073643d3",
   "metadata": {},
   "outputs": [],
   "source": []
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "b5a6f46a-ff3a-4b98-980a-b3d5a688c231",
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "environment": {
   "kernel": "python3",
   "name": "pytorch-gpu.1-13.m103",
   "type": "gcloud",
   "uri": "gcr.io/deeplearning-platform-release/pytorch-gpu.1-13:m103"
  },
  "kernelspec": {
   "display_name": "Python 3 (ipykernel)",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.7.12"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}
